<a href="https://colab.research.google.com/github/yota-nagai/combinatorial-optimization-problems/blob/main/14_%E3%82%B0%E3%83%A9%E3%83%95%E5%90%8C%E5%9E%8B%E6%80%A7%E5%88%A4%E5%AE%9A%E5%95%8F%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q amplify
from amplify import VariableGenerator, FixstarsClient, solve, Solver
from datetime import timedelta

client = FixstarsClient()
client.token = "******************"  #アクセストークンを入力
client.parameters.timeout = timedelta(milliseconds=2000)  # タイムアウトは 2000 ms


#################### グラフ同型性判定問題 ####################

N1,M1=map(int,input().split())  #グラフG1の頂点数、辺の数
edge1=set()  #グラフG1の辺集合（無向グラフ）
for _ in range(M1):
  u1,v1=map(int,input().split())
  u1-=1
  v1-=1
  edge1.add((u1,v1))
  edge1.add((v1,u1))  #辺リストの巡回ではなく、全頂点ペアの総当たりをしていく

N2,M2=map(int,input().split())  #グラフG2の頂点数、辺の数  N1=N2が前提
edge2=set()  #グラフG2の辺集合（無向グラフ）
for _ in range(M1):
  u2,v2=map(int,input().split())
  u2-=1
  v2-=1
  edge2.add((u2,v2))
  edge2.add((v2,u2))  #辺リストの巡回ではなく、全頂点ペアの総当たりをしていく

#バイナリ変数の設定
gen=VariableGenerator()
x_vi=gen.array("Binary",shape=(N,N))  #G2内の頂点vがG1内の頂点iに対応するかどうか

#制約項①　G1の頂点とG2の頂点が、もれや重複なく1対1対応
f1=0
for i in range(N1):
  cnt=0
  for v in range(N2):
    cnt+=x_vi[v][i]
  f1+=(1-cnt)**2
for v in range(N2):
  cnt=0
  for i in range(N1):
    cnt+=x_vi[v][i]
  f1+=(1-cnt)**2

#制約項②　G1とG2で、隣接状況が一致
f2=0
for i in range(N1):
  for j in range(N1):
    if (i,j) in edge1:
      for u in range(N2):
        for v in  range(N2):
          if (u,v) not in edge2:
            f2+=x_vi[u][i]*x_vi[v][j]
    else:
      for u in range(N2):
        for v in range(N2):
          if (u,v) in edge:
            f2+=x_vi[u][i]*x_vi[v][j]

#ペナルティ係数の設定
A,B=map(int,input().split())
f=A*f1+B*f2


#ソルバー実行、最適解の取得
result=solve(f,client)
objective=result.best.objective
values=result.best.values


if objective>0:
  print("G1とG2は同型でない")
else:
  print("G1とG2は同型")
  x_vi_values=x_vi.evaluate(values)
  for v in range(N2):
    for i in range(N1):
      if x_vi_values[v][i]==1:
        print(f"G1の頂点{i+1}とG2の頂点{v+1}が対応")